In [1]:
import time
import datetime
import os
import yaml
import math
import sys
import gurobipy
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum

print(f"python version: {sys.version}")
print(f"gurobipy version: {gurobipy.gurobi.version()}")


python version: 3.9.18 (main, Sep 11 2023, 08:38:23) 
[Clang 14.0.6 ]
gurobipy version: (11, 0, 0)


In [195]:
# 添加相對路徑到 sys.path
sys.path.append('../')  
from path_config import INSTANCES_DIR
def load_specific_yaml(filename):
    """
    加載指定的 YAML 檔案。
    
    Parameters:
    filename (str): 在 instances 資料夾中的 YAML 檔案名。
    
    Returns:
    dict: YAML 檔案內容。
    """
    file_path = os.path.join(INSTANCES_DIR, filename)
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)
    return data

# Load experiments yaml
config = load_specific_yaml('instance_F_0.yaml')
config['lambda_for_G'] = 6e-6
config['M'] = 1000
config

{'A_opponent_bar': [52],
 'B': [[2], [2], [3]],
 'C': [3, 3, 1],
 'D': [[88.55, 75.13, 17.12], [34.93, 19.24, 76.54]],
 'D_comp': [[51.61], [63.13]],
 'F': [5, 8, 5],
 'H': [23, 21],
 'M': 1000,
 'U_L': [10, 8, 8],
 'U_LT': [[3], [1], [3]],
 'U_T': [7],
 'V': [[1289], [1678], [1786]],
 'i_amount': 2,
 'j_amount': 3,
 'k_amount': 1,
 'l_amount': 1,
 'lambda_for_G': 6e-06}

In [ ]:
I = set(range(0, config['i_amount']))  
J = set(range(0, config['j_amount']))  
K = set(range(0, config['k_amount']))  
L = set(range(0, config['l_amount']))  
'''
Decision Variables
===============================
y : Whether location j build facility or not, Dim: (j) , BINARY
x : The amount of resource k allocated to location j, Dim: (j,k), CONTINUOUS
aX : Extra attractiveness allocated to location j, Dim: (j), CONTINUOUS
===============================

Other Variables
u : Utility of facility j to customer point i, Dim:(i, j), CONTINUOUS
P : Probability of customer i choosing facility j, Dim : (i, j), CONTINUOUS
TA : Total Attractiveness for customer location i, Dim : (i), CONTINUOUS
===============================
'''

'\nDecision Variables\n===============================\ny : Whether location j build facility or not, Dim: (j) , BINARY\nx : The amount of resource k allocated to location j, Dim: (j,k), CONTINUOUS\naX : Extra attractiveness allocated to location j, Dim: (j), CONTINUOUS\n===============================\n\nOther Variables\nu : Utility of facility j to customer point i, Dim:(i, j), CONTINUOUS\nP : Probability of customer i choosing facility j, Dim : (i, j), CONTINUOUS\nTA : Total Attractiveness for customer location i, Dim : (i), CONTINUOUS\n===============================\n'

In [198]:
# Initialize the model
model = Model("Competitive Facility Location")
model.setParam('NumericFocus', 3)
y = model.addVars(J, vtype=GRB.BINARY, name="y") #3.14
x = model.addVars(J, K, vtype=GRB.CONTINUOUS, name="x", lb=0) # 3.12
aX = model.addVars(J, vtype=GRB.CONTINUOUS, name="aX", lb=0) # 3.13

u = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="utility(u)", lb=0) # utility var(3.5)
P = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="P, utility vs TA ratio", lb=0.0, ub=1.0) # Probability of i choosing j
TA = model.addVars(I, vtype=GRB.CONTINUOUS, name="TA", lb=0.00000000)
TAi_lambda = model.addVars(I, vtype=GRB.CONTINUOUS, name="negative TAi*lambda(e's power)", lb= -100000, ub= -0.000001) #-lambda*TAi
G_exp_vars = model.addVars(I, vtype=GRB.CONTINUOUS, name="G_exp_vars", lb=0.0, ub=1) #e^-(lambda*TAi), ub=0.5

model.update()

model.addConstrs((x[j, k] <= config['U_LT'][j][k] * y[j] for j in J for k in K), "(3.1) Resource only for built facility, and amount of type k for facility j is limit at U_LT[j, k]")
model.addConstrs((quicksum(x[j, k] for j in J) <= config['U_T'][k] for k in K), "(3.2) The total amount of resource k is U_T[k]")
model.addConstrs((quicksum(x[j, k] for k in K) <= config['U_L'][j] for j in J), "(3.3) The max resource sum for facility j is U_L[j]")
model.addConstrs((aX[j] <= config['M'] * y[j] for j in J), "(3.4) Extra Attractiveness Limit")

model.addConstrs((u[i, j] == ((quicksum(config['V'][j][k] * x[j, k] for k in K) + aX[j]) / (config['D'][i][j]**2)) for i in I for j in J), "(3.5) utility for facility j to customer i")
model.addConstrs((u[i, j] == P[i, j] * TA[i] for i in I for j in J), "(3.8) calculate P by uij/TAi, 分母要是常數所以用乘的")
## TA的限制式, 因為不能讓 G_exp中 addGenConstrExp內的變數為多個變數的線性組合所以要另外創TA變數
model.addConstrs((TA[i] == quicksum(u[i, j] for j in J) + (quicksum(config['A_opponent_bar'][l] / (config['D_comp'][i][l]**2) for l in L)) for i in I), "(3.7) calculate total TA by u and A_bar")

## G的計算
# def G(tai):
#     return 1 - np.exp(-config['lambda_for_G'] * tai)
model.addConstrs((TAi_lambda[i] == (-config['lambda_for_G'] * TA[i]) for i in I), "negative tai*lambda constr")
# Gurobi無法直接在目標式有exponential
for i in I:
    model.addGenConstrExpA(TAi_lambda[i], G_exp_vars[i], math.e, name=f"G_exp_constr_{i}")



# Objective function: Maximize profit by attracting customers - costs
objective = ((quicksum(config['H'][i] * (1 - G_exp_vars[i]) * (quicksum(P[i, j] for j in J)) for i in I)) \
          - (quicksum(config['F'][j] * y[j] + config['C'][j] * aX[j] + quicksum(config['B'][j][k] * x[j, k] for k in K) for j in J)))
model.setObjective(objective, GRB.MAXIMIZE)


# Solve and Output
model.optimize()
if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
else:
    print("No optimal solution found.")
    model.computeIIS()
    # 将IIS输出到文件
    # model.write("model_iis.mps")
    print("IIS written to file 'model_iis.mps'")



Set parameter NumericFocus to value 3
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 21.5.0 21F79)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 20 rows, 27 columns and 48 nonzeros
Model fingerprint: 0x233d6721
Model has 6 quadratic objective terms
Model has 6 quadratic constraints
Model has 2 general constraints
Variable types: 24 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-06, 1e+03]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]


  QObjective range [4e+01, 5e+01]
  Bounds range     [1e-06, 1e+05]
  RHS range        [1e-02, 1e+01]
Presolve added 0 rows and 53 columns
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 56 rows, 88 columns, 288 nonzeros
Presolved model has 2 SOS constraint(s)
Presolved model has 12 bilinear constraint(s)
         in product terms.
         Presolve was not able to compute smaller bounds for these variables.
         Consider bounding these variables or reformulating the model.


Solving non-convex MIQCP

Variable types: 85 continuous, 3 integer (3 binary)
Found heuristic solution: objective -6.5112575

Root relaxation: objective -4.289450e-01, 33 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -0.42895    0    6   -6.51126   -0.42895  93.4%     -    0s
     0     2   -3.28186    0    6   -6.51126   -3.28

In [199]:
for var in model.getVars():
    print(f"{var.varName} = {var.x}")
print("")

y[0] = 0.0
y[1] = 0.0
y[2] = 1.0
x[0,0] = 0.0
x[1,0] = 0.0
x[2,0] = 0.5038955107867301
aX[0] = 0.0
aX[1] = 0.0
aX[2] = 0.0
utility(u)[0,0] = 0.0
utility(u)[0,1] = 0.0
utility(u)[0,2] = 3.0705376229185997
utility(u)[1,0] = 0.0
utility(u)[1,1] = 0.0
utility(u)[1,2] = 0.1536190333618907
P, utility vs TA ratio[0,0] = 0.0
P, utility vs TA ratio[0,1] = 0.0
P, utility vs TA ratio[0,2] = 0.9936821589584729
P, utility vs TA ratio[1,0] = 0.0
P, utility vs TA ratio[1,1] = 0.0
P, utility vs TA ratio[1,2] = 0.921714196459699
TA[0] = 3.09006013163906
TA[1] = 0.16666666666666666
negative TAi*lambda(e's power)[0] = -1.854036078983436e-05
negative TAi*lambda(e's power)[1] = -1.0000000040269115e-06
G_exp_vars[0] = 0.9999820757689817
G_exp_vars[1] = 0.9999989990208253



# References

## G calculation substitude exponential

https://www.gurobi.com/documentation/10.0/refman/py_model_agc_exp.html

https://support.gurobi.com/hc/en-us/community/posts/360077178491-Build-an-objective-function-with-Log-and-Exponential-inside-

https://support.gurobi.com/hc/en-us/community/posts/10481250723217-How-to-add-log-and-exponential-term-in-objective-function
